# 5-bp randomization

In [40]:
label_file = pd.read_table('/casa/haedong/data/dicer_motif_5_bp_window_wt_dicer_aqseq_haedong/labels.txt'.format(DATA_DIR))
label_file

,filepath,label,group,shortname,5_adapter,3_adapter,cell_line,description
0,raw_files/sample_pre-let-7a-1_initial_1.fq.gz,let7a1_initial,let7a1,7a1_i,GTTCAGAGTTCTACAGTCCGACGATC,TGGAATTCTCGGGTGCCAAGG,synthetic RNA,NaN
1,raw_files/sample_pre-let-7a-1_uncleaved_1.fq.gz,let7a1_uncleaved,let7a1,7a1_uc,GTTCAGAGTTCTACAGTCCGACGATC,TGGAATTCTCGGGTGCCAAGG,synthetic RNA,NaN


In [41]:
dataframes = []
for label in label_file['label']:
    count = pd.read_table('{}/processed_files/{}.count.txt'.format(DATA_DIR, label))
    count.loc[:, 'barcode'] = count['barcode_5'] + '/' + count['barcode_3']
    count = count.set_index('barcode')
    count = count.loc[:, count.columns.str.startswith('count')]
    count.columns = ['count_{}'.format(label.split('_')[1].replace('initial', 'input'))]
    dataframes.append(count)
    
count = pd.concat(dataframes, axis=1).fillna(0)
count.index = count.index.str.replace('T', 'U')
count = count.loc[~count.index.str.contains('N')]
count

/casa/haedong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


,count_input,count_uncleaved
AAAAA/AAAAA,1.0,0.0
AAAAA/AAAAG,1.0,0.0
AAAAA/AAACC,2.0,0.0
AAAAA/AAACG,0.0,1.0
AAAAA/AAAGC,0.0,1.0
...,...,...
UUUUU/UUUGU,75.0,55.0
UUUUU/UUUUA,50.0,23.0
UUUUU/UUUUC,107.0,48.0
UUUUU/UUUUG,53.0,25.0


In [42]:
df = count + 1
score = pd.DataFrame((df['count_input'] / df['count_input'].sum()) / (df['count_uncleaved'] / df['count_uncleaved'].sum()))
score.columns = ['cleavage_score']
score = score.loc[score.sort_values('cleavage_score', ascending=False).index]
score

,cleavage_score
UCACA/AUCUG,36.653844
UCCCC/GGCGA,34.949014
UCCGC/GCCGA,34.949014
AGAUA/GUGGC,32.391769
GAAAU/GUGGC,32.391769
...,...
AAUAA/GGACA,0.077492
AUUCC/AAAAA,0.071035
AGUCG/GAGAA,0.071035
CCCUA/GACGA,0.071035


In [43]:
summary = pd.concat([count, score], axis=1)
summary

/casa/haedong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,count_input,count_uncleaved,cleavage_score
AAAAA/AAAAA,1.0,0.0,1.704830
AAAAA/AAAAG,1.0,0.0,1.704830
AAAAA/AAACC,2.0,0.0,2.557245
AAAAA/AAACG,0.0,1.0,0.426207
AAAAA/AAAGC,0.0,1.0,0.426207
...,...,...,...
UUUUU/UUUGU,75.0,55.0,1.156849
UUUUU/UUUUA,50.0,23.0,1.811382
UUUUU/UUUUC,107.0,48.0,1.878792
UUUUU/UUUUG,53.0,25.0,1.770400


In [49]:
summary_abundant = summary.loc[summary['count_input'] > 100].sort_values('cleavage_score', ascending=False)
summary_not_abundant = summary.loc[summary['count_input'] <= 100].sort_values(['count_input', 'cleavage_score'], ascending=[False, False])
summary = pd.concat([summary_abundant, summary_not_abundant])
summary

,count_input,count_uncleaved,cleavage_score
AGCAC/GUCCU,107.0,4.0,18.412163
AACGC/GCCUU,138.0,8.0,13.165076
UUAGC/GCUAG,107.0,6.0,13.151545
UACGC/GCCUA,121.0,7.0,12.999328
AACCU/AGCUU,114.0,7.0,12.253465
...,...,...,...
UCUGA/UAAAA,0.0,10.0,0.077492
AGUCG/GAGAA,0.0,11.0,0.071035
AUUCC/AAAAA,0.0,11.0,0.071035
CCCUA/GACGA,0.0,11.0,0.071035


In [50]:
summary.to_csv('/casa/haedong/project/dicer_motif/for_publication/cleavage_score.minus1_to_3_randomization.txt', sep='\t', header=True, index=True)

# 3-bp randomization (-1-to-1 window)

In [1]:
RNA2DIR2DILUTIONS = {
    'let7a1': {
        '/casa/haedong/data/dicer_motif_3_bp_window_wt_dicer_aqseq_haedong/altering_time_rna_dicer_1_2_ratio_randomized_2/let_7a_1': [10, 20, 30]
    },
    
    'mir374b': {
        '/casa/haedong/data/dicer_motif_3_bp_window_wt_dicer_aqseq_haedong/altering_time_rna_dicer_1_2_ratio_randomized_2/mir_374b': [10, 20, 30]
    }
}

In [2]:
dataframes_count = []
dataframes_score = []
for rna, dir2dilutions in RNA2DIR2DILUTIONS.items():
    for data_dir, dilutions in dir2dilutions.items():
        initial = pd.read_table('{}/processed_files/{}_2_input.count.txt'.format(data_dir, rna))
        initial.loc[:, 'barcode'] = initial['barcode_5'] + '/' + initial['barcode_3']
        initial = initial.set_index('barcode')
        initial = initial.loc[~initial.index.str.contains('N')]
        initial = initial[['count_input']]
        initial.columns = ['count_{}_input'.format(rna)]
        dataframes_count.append(initial)

        for dilution in dilutions:
            uncleaved = pd.read_table('{}/processed_files/{}_2_{}.count.txt'.format(data_dir, rna, dilution))
            uncleaved.loc[:, 'barcode'] = uncleaved['barcode_5'] + '/' + uncleaved['barcode_3']
            uncleaved = uncleaved.set_index('barcode')
            uncleaved = uncleaved.loc[~uncleaved.index.str.contains('N')]
            uncleaved = uncleaved[['count_{}'.format(dilution)]]
            uncleaved.columns = ['count_{}_{}'.format(rna, dilution)]
            dataframes_count.append(uncleaved)

            count = pd.concat([initial, uncleaved], axis=1).fillna(0).astype(int)
            count = count + 1

            # cleavage score
            score = pd.DataFrame((count['count_{}_input'.format(rna)] / count['count_{}_input'.format(rna)].sum()) / \
                                 (count['count_{}_{}'.format(rna, dilution)] / count['count_{}_{}'.format(rna, dilution)].sum()))
            score.columns = ['cleavage_score_{}_{}'.format(rna, dilution)]
            dataframes_score.append(score)

/casa/haedong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [3]:
count = pd.concat(dataframes_count, axis=1).fillna(0)
score = pd.concat(dataframes_score, axis=1)

summary = pd.concat([count, score], axis=1)
summary.index = [motif.replace('T', 'U') for motif in summary.index]
summary.columns = [col.replace('30rerun', '30') for col in summary.columns]
summary.loc[:, 'cleavage_score_average_10'] = summary[['cleavage_score_let7a1_10', 'cleavage_score_mir374b_10']].mean(1)
summary.loc[:, 'cleavage_score_average_20'] = summary[['cleavage_score_let7a1_20', 'cleavage_score_mir374b_20']].mean(1)
summary.loc[:, 'cleavage_score_average_30'] = summary[['cleavage_score_let7a1_30', 'cleavage_score_mir374b_30']].mean(1)
summary = summary.sort_values('cleavage_score_average_20', ascending=False)
summary

/casa/haedong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,count_let7a1_input,count_let7a1_10,count_let7a1_20,count_let7a1_30,count_mir374b_input,count_mir374b_10,count_mir374b_20,count_mir374b_30,cleavage_score_let7a1_10,cleavage_score_let7a1_20,cleavage_score_let7a1_30,cleavage_score_mir374b_10,cleavage_score_mir374b_20,cleavage_score_mir374b_30,cleavage_score_average_10,cleavage_score_average_20,cleavage_score_average_30
CGC/GCG,1055,263,82,45,212,31,11,6,4.358202,15.199809,25.705927,5.141645,16.937334,29.181761,4.749923,16.068571,27.443844
AGC/GCA,2098,561,179,82,3948,365,213,184,4.069335,13.931319,28.317931,8.334480,17.608406,20.471296,6.201908,15.769863,24.394614
CGC/GCC,1232,168,90,57,4008,433,273,211,7.949205,16.187284,23.804675,7.135407,13.961503,18.135525,7.542306,15.074394,20.970100
CAC/GUC,1026,169,71,32,2215,204,173,115,6.582166,17.040811,34.848467,8.350036,12.152544,18.320685,7.466101,14.596677,26.584576
AAU/AUU,100,35,9,7,613,78,37,33,3.056794,12.066288,14.137043,6.003627,15.418121,17.318863,4.530211,13.742205,15.727953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCU/UAU,623,1000,1351,1583,1746,1918,2649,3223,0.679200,0.551392,0.441120,0.703219,0.629062,0.519670,0.691209,0.590227,0.480395
CUC/UGU,795,1363,1847,2026,1959,1826,2855,3129,0.635837,0.514592,0.439730,0.828686,0.654854,0.600540,0.732262,0.584723,0.520135
CUC/UCU,673,997,1450,1600,2178,2467,3446,3874,0.735829,0.554938,0.471407,0.682000,0.603202,0.539281,0.708914,0.579070,0.505344
CCU/UUU,601,870,1266,1339,2241,2539,3693,4362,0.753053,0.567639,0.503059,0.681827,0.579142,0.492811,0.717440,0.573391,0.497935


In [11]:
summary.loc[:, 'gym_score'] = summary['cleavage_score_average_20'] / summary['cleavage_score_average_20'].max() * 100
summary

,count_let7a1_input,count_let7a1_10,count_let7a1_20,count_let7a1_30,count_mir374b_input,count_mir374b_10,count_mir374b_20,count_mir374b_30,cleavage_score_let7a1_10,cleavage_score_let7a1_20,cleavage_score_let7a1_30,cleavage_score_mir374b_10,cleavage_score_mir374b_20,cleavage_score_mir374b_30,cleavage_score_average_10,cleavage_score_average_20,cleavage_score_average_30,gym_score
CGC/GCG,1055,263,82,45,212,31,11,6,4.358202,15.199809,25.705927,5.141645,16.937334,29.181761,4.749923,16.068571,27.443844,100.000000
AGC/GCA,2098,561,179,82,3948,365,213,184,4.069335,13.931319,28.317931,8.334480,17.608406,20.471296,6.201908,15.769863,24.394614,98.141037
CGC/GCC,1232,168,90,57,4008,433,273,211,7.949205,16.187284,23.804675,7.135407,13.961503,18.135525,7.542306,15.074394,20.970100,93.812906
CAC/GUC,1026,169,71,32,2215,204,173,115,6.582166,17.040811,34.848467,8.350036,12.152544,18.320685,7.466101,14.596677,26.584576,90.839919
AAU/AUU,100,35,9,7,613,78,37,33,3.056794,12.066288,14.137043,6.003627,15.418121,17.318863,4.530211,13.742205,15.727953,85.522255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCU/UAU,623,1000,1351,1583,1746,1918,2649,3223,0.679200,0.551392,0.441120,0.703219,0.629062,0.519670,0.691209,0.590227,0.480395,3.673177
CUC/UGU,795,1363,1847,2026,1959,1826,2855,3129,0.635837,0.514592,0.439730,0.828686,0.654854,0.600540,0.732262,0.584723,0.520135,3.638925
CUC/UCU,673,997,1450,1600,2178,2467,3446,3874,0.735829,0.554938,0.471407,0.682000,0.603202,0.539281,0.708914,0.579070,0.505344,3.603744
CCU/UUU,601,870,1266,1339,2241,2539,3693,4362,0.753053,0.567639,0.503059,0.681827,0.579142,0.492811,0.717440,0.573391,0.497935,3.568399


In [14]:
summary.to_csv('/casa/haedong/project/dicer_motif/for_publication/cleavage_score.minus1_to_1_randomization.txt', sep='\t', header=True, index=True)

# 3-bp randomization (1-to-3 window)

In [6]:
RNA2DIR2DILUTIONS = {
    'let7a1': {
        '/casa/haedong/data/dicer_motif_3_bp_window_wt_dicer_aqseq_haedong/altering_time_rna_dicer_1_2_ratio_randomized_1/pre_let_7a_1': [10, 20, 30]
    },
    
    'mir374b': {
        '/casa/haedong/data/dicer_motif_3_bp_window_wt_dicer_aqseq_haedong/altering_time_rna_dicer_1_2_ratio_randomized_1/pre_mir_374b': [10, 20, 30]
    }
}

In [7]:
dataframes_count = []
dataframes_score = []
for rna, dir2dilutions in RNA2DIR2DILUTIONS.items():
    for data_dir, dilutions in dir2dilutions.items():
        initial = pd.read_table('{}/processed_files/{}_1_input.count.txt'.format(data_dir, rna))
        initial.loc[:, 'barcode'] = initial['barcode_5'] + '/' + initial['barcode_3']
        initial = initial.set_index('barcode')
        initial = initial.loc[~initial.index.str.contains('N')]
        initial = initial[['count_input']]
        initial.columns = ['count_{}_input'.format(rna)]
        dataframes_count.append(initial)

        for dilution in dilutions:
            uncleaved = pd.read_table('{}/processed_files/{}_1_{}.count.txt'.format(data_dir, rna, dilution))
            uncleaved.loc[:, 'barcode'] = uncleaved['barcode_5'] + '/' + uncleaved['barcode_3']
            uncleaved = uncleaved.set_index('barcode')
            uncleaved = uncleaved.loc[~uncleaved.index.str.contains('N')]
            uncleaved = uncleaved[['count_{}'.format(dilution)]]
            uncleaved.columns = ['count_{}_{}'.format(rna, dilution)]
            dataframes_count.append(uncleaved)

            count = pd.concat([initial, uncleaved], axis=1).fillna(0).astype(int)
            count = count + 1

            # cleavage score
            score = pd.DataFrame((count['count_{}_input'.format(rna)] / count['count_{}_input'.format(rna)].sum()) / \
                                 (count['count_{}_{}'.format(rna, dilution)] / count['count_{}_{}'.format(rna, dilution)].sum()))
            score.columns = ['cleavage_score_{}_{}'.format(rna, dilution)]
            dataframes_score.append(score)

/casa/haedong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [8]:
count = pd.concat(dataframes_count, axis=1).fillna(0)
score = pd.concat(dataframes_score, axis=1)

summary = pd.concat([count, score], axis=1)
summary.index = [motif.replace('T', 'U') for motif in summary.index]
summary.columns = [col.replace('30rerun', '30') for col in summary.columns]
summary.loc[:, 'cleavage_score_average_10'] = summary[['cleavage_score_let7a1_10', 'cleavage_score_mir374b_10']].mean(1)
summary.loc[:, 'cleavage_score_average_20'] = summary[['cleavage_score_let7a1_20', 'cleavage_score_mir374b_20']].mean(1)
summary.loc[:, 'cleavage_score_average_30'] = summary[['cleavage_score_let7a1_30', 'cleavage_score_mir374b_30']].mean(1)
summary = summary.sort_values('cleavage_score_average_20', ascending=False)
summary

/casa/haedong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,count_let7a1_input,count_let7a1_10,count_let7a1_20,count_let7a1_30,count_mir374b_input,count_mir374b_10,count_mir374b_20,count_mir374b_30,cleavage_score_let7a1_10,cleavage_score_let7a1_20,cleavage_score_let7a1_30,cleavage_score_mir374b_10,cleavage_score_mir374b_20,cleavage_score_mir374b_30,cleavage_score_average_10,cleavage_score_average_20,cleavage_score_average_30
GCC/CGC,1037,661,262,178,2899,1502,605,200,2.249329,3.941487,8.087021,1.944341,4.095357,11.019544,2.096835,4.018422,9.553283
CAC/CUG,573,378,152,116,1626,795,333,139,2.172633,3.746614,6.841799,2.059718,4.168766,8.876080,2.116176,3.957690,7.858939
GAC/CUC,549,413,146,119,1089,554,236,65,1.905794,3.736490,6.391837,1.979096,3.935904,12.613751,1.942445,3.836197,9.502794
UUC/CAG,374,332,178,175,2285,960,415,189,1.615477,2.092169,2.971412,2.397100,4.702722,9.189340,2.006289,3.397445,6.080376
ACC/CGU,481,311,144,98,1417,879,387,170,2.216186,3.319690,6.789786,1.623779,3.127598,6.333472,1.919982,3.223644,6.561629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGA/GGC,684,1326,1110,1701,1807,2254,2356,2462,0.740514,0.615737,0.561274,0.807951,0.656455,0.560655,0.774233,0.636096,0.560965
ACU/CUC,501,1047,854,1552,2984,3491,3724,3806,0.687157,0.586349,0.450792,0.861397,0.685779,0.598857,0.774277,0.636064,0.524825
CCC/ACC,301,511,501,845,2781,3468,3615,4058,0.846156,0.600789,0.497830,0.808139,0.658408,0.523480,0.827148,0.629598,0.510655
CCA/ACC,585,1111,978,1587,3479,4332,4576,5047,0.755973,0.597769,0.514626,0.809327,0.650676,0.526528,0.782650,0.624222,0.520577


In [9]:
summary.to_csv('/casa/haedong/project/dicer_motif/for_publication/cleavage_score.1_to_3_randomization.txt', sep='\t', header=True, index=True)